In [2]:
#!/usr/bin/env python
"""
Full-Extent Reprojection with MontagePy’s mProjExec
"""

import numpy as np
from pathlib import Path
from astropy.io import fits
import shutil

# MontagePy executive functions
from MontagePy.main import mGetHdr, mImgtbl, mProjExec

# ----------------------------------------------------------------------
# Utility to retrieve MontagePy version (no __version__ in bindings)
# ----------------------------------------------------------------------
try:
    from importlib.metadata import version, PackageNotFoundError
except ImportError:
    from importlib_metadata import version, PackageNotFoundError

def reproject_with_mProjExec(
    multi_band_fits: Path,
    reference_fits: Path,
    work_dir: Path = Path("montagepy_work")
) -> np.ndarray:
    """
    1. Splits a multi-band FITS into separate 2D FITS files.
    2. Extracts the template header via mGetHdr.
    3. Builds an image metadata table with mImgtbl.
    4. Runs mProjExec(..., wholeImages=True) to reproject all images
       across the full template area (no cropping).
    5. Loads and stacks the reprojected FITS into a NumPy array.
    Returns:
        cube : np.ndarray of shape (n_images, 438, 437)
    """
    # --- 1. Prepare working directories and split bands ------------
    if work_dir.exists():
        shutil.rmtree(work_dir)
    work_dir.mkdir()
    input_dir = work_dir / "legacy_fits"
    input_dir.mkdir()
    proj_dir = work_dir / "proj"
    proj_dir.mkdir()

    # Load and split the multi-band FITS into standalone 2D files
    with fits.open(multi_band_fits) as hdul:
        data = hdul[0].data
        hdr  = hdul[0].header

    # If >20 bands on first axis, move spectral axis to front
    if data.ndim > 2 and data.shape[0] > 20:
        data = np.moveaxis(data, -1, 0)

    # Write each 2D slice as its own file
    for idx in range(data.shape[0] if data.ndim > 2 else 1):
        band = data[idx] if data.ndim > 2 else data
        band_hdr = hdr.copy()
        band_hdr["NAXIS"] = 2
        # Remove any 3D keywords
        for key in list(band_hdr):
            if key.startswith(("NAXIS3","CTYPE3","CUNIT3",
                               "CRVAL3","CRPIX3","CDELT3",
                               "CD3_","PC3_")):
                del band_hdr[key]
        out_path = input_dir / f"band_{idx:02d}.fits"
        fits.PrimaryHDU(band.astype(np.float64), header=band_hdr)\
            .writeto(out_path, overwrite=True)

    # --- 2. Create a 2D template.fits with proper WCS header --------
    # Extract the FLUX extension header from reference_fits
    with fits.open(reference_fits) as hd:
        flux_hdr = hd["FLUX"].header
        ny, nx   = hd["FLUX"].data.shape

    template_fits = work_dir / "template.fits"
    fits.PrimaryHDU(
        data=np.zeros((ny, nx), dtype=np.float32),
        header=flux_hdr
    ).writeto(template_fits, overwrite=True)

    # Extract to ASCII .hdr
    hdr_file = work_dir / "template.hdr"
    res_hdr = mGetHdr(str(template_fits), str(hdr_file))
    if res_hdr.get("status") != "0":
        raise RuntimeError(f"mGetHdr failed: {res_hdr}")  # 

    # --- 3. Build image metadata table -----------------------------
    tbl_file = work_dir / "images.tbl"
    res_tbl  = mImgtbl(str(input_dir), str(tbl_file))
    if res_tbl.get("status") != "0":
        raise RuntimeError(f"mImgtbl failed: {res_tbl}")  # 

    # --- 4. Reproject with full-extent (no cropping) ----------------
    res_proj = mProjExec(
        str(input_dir),
        str(tbl_file),
        str(hdr_file),
        str(proj_dir),
        wholeImages=True        # Force full-extent outputs  
    )
    if res_proj.get("status") != "0":
        raise RuntimeError(f"mProjExec failed: {res_proj}")  # 

    # --- 5. Load and stack results ---------------------------------
    fits_files = sorted(proj_dir.glob("*.fits"))
    cube       = []
    for fpath in fits_files:
        arr = fits.getdata(str(fpath))
        # assert arr.shape == (ny, nx), (
        #     f"Unexpected shape {arr.shape} for {fpath.name}"
        # )
        cube.append(arr)

    return np.stack(cube, axis=0)

if __name__ == "__main__":
    # Version check
    try:
        mp_ver = version("MontagePy")
        print(f"MontagePy version: {mp_ver}")  # 
    except PackageNotFoundError:
        print("MontagePy not found in this environment")

    multi_band  = Path("IC3392_DESI_grih.fits")
    reference   = Path("IC3392_SPATIAL_BINNING_maps.fits")

    cube = reproject_with_mProjExec(
        multi_band_fits=multi_band,
        reference_fits=reference
    )
    print(f"\nReprojection complete: stacked cube shape = {cube.shape}")


MontagePy version: 2.2.0

Reprojection complete: stacked cube shape = (8, 3931, 3931)


In [2]:
from MontagePy.main import mProject, mViewer

In [3]:
from importlib.metadata import version, PackageNotFoundError

try:
    mp_ver = version("MontagePy")
    print("MontagePy version:", mp_ver)
except PackageNotFoundError:
    print("MontagePy is not installed")


MontagePy version: 2.2.0


In [4]:
import pkg_resources

mp_ver = pkg_resources.get_distribution("MontagePy").version
print("MontagePy version:", mp_ver)


MontagePy version: 2.2.0


/var/folders/rm/clp89v210v32vh5vjt2kx7b80000gr/T/ipykernel_62639/55126941.py:1: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources
